In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI 

model = ChatOpenAI()

In [7]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()

In [3]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader("https://blog.langchain.dev/langgraph/")

docs = loader.load()

In [4]:
from langchain_openai import OpenAIEmbeddings 
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)

In [5]:
documents 

[Document(page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n                LangChain Blog\n        \n\n\n\n\n\n\nHome\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\nCase Studies\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024', metadata={'source': 'https://blog.langchain.dev/langgraph/', 'title': 'LangGraph', 'language': 'en'}),
 Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primaril

In [8]:
from langchain_community.vectorstores import FAISS 

vector = FAISS.from_documents(documents, embeddings)

In [9]:
retriever = vector.as_retriever()

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search\
 query to look up in order to get information relevant to the conversaion")
    ]
)

In [11]:
from langchain.chains import create_history_aware_retriever

retriver_chain=create_history_aware_retriever(
    model, retriever, prompt
)

In [12]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history=[
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!")
]

In [13]:
retriver_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how",
    }
)

[Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\'ve heard requests for, and examples of implementing those. We will finish with a preview of what we will be releasing next.MotivationOne of the big v

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the user's questions based on the below\
 context: \n\n{context}"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")
    ]
)

In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain

documents_chain = create_stuff_documents_chain(model, prompt)

In [17]:
from langchain.chains import create_retrieval_chain

retriver_chain = create_retrieval_chain(
    retriver_chain, documents_chain
)

In [18]:
chat_history=[
    HumanMessage(
        content="Can I use LangGraph for agent runtimes?"
    ),
    AIMessage(content="Yes!")
]

In [19]:
response = retriver_chain.invoke(
    {
        "chat_history": chat_history,
        "input": "Tell me how",
    }
)

In [20]:
response 

{'chat_history': [HumanMessage(content='Can I use LangGraph for agent runtimes?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='TL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through the motivations for LangGraph. We will then cover the basic functionality it provides. We will then spotlight two agent runtimes we\'ve implemented already. We will then highlight a few of the common modifications to these runtimes we\

In [21]:
response["answer"]

'LangGraph is a module built on top of LangChain specifically designed to enable the creation of cyclical graphs, which are often needed for agent runtimes. LangGraph is completely interoperable with the LangChain ecosystem, meaning you can use it alongside LangChain to build and customize agent runtimes.\n\nTo use LangGraph for agent runtimes, you first need to create a StateGraph object, which represents the graph. This object is initialized with a state definition, which represents a central state object that is updated over time. Nodes in the graph update the attributes of this state object.\n\nYou can add nodes to the graph using the `graph.add_node(name, value)` syntax. The name parameter is a string used to refer to the node when adding edges, and the value parameter is a function or LCEL runnable that will be called. This function/LCEL should accept a dictionary representing the state object as input and output a dictionary with keys of the state object to update.\n\nAfter addi

'LangGraph is a module built on top of LangChain specifically designed to enable the creation of cyclical graphs, which are often needed for agent runtimes. LangGraph is completely interoperable with the LangChain ecosystem, meaning you can use it alongside LangChain to build and customize agent runtimes.\n\nTo use LangGraph for agent runtimes, you first need to create a StateGraph object, which represents the graph. This object is initialized with a state definition, which represents a central state object that is updated over time. Nodes in the graph update the attributes of this state object.\n\nYou can add nodes to the graph using the `graph.add_node(name, value)` syntax. The name parameter is a string used to refer to the node when adding edges, and the value parameter is a function or LCEL runnable that will be called. This function/LCEL should accept a dictionary representing the state object as input and output a dictionary with keys of the state object to update.\n\nAfter adding nodes, you can add edges to connect them and create the graph. There are different types of edges, such as the starting edge, normal edges, and conditional edges. The starting edge connects the start of the graph to a specific node. Normal edges define the order in which nodes should be called. Conditional edges use a function (often powered by an LLM) to determine which node to go to next based on the output of the previous node.\n\nOnce you have defined your graph, you can compile it into a runnable using the `graph.compile()` method. This runnable can be invoked and called in the same manner as a LangChain chain.\n\nAdditionally, LangGraph provides a way to recreate the LangChain AgentExecutor, allowing you to use existing LangChain agents but with the ability to modify the internals of the AgentExecutor.\n\nOverall, LangGraph provides the necessary functionality for creating cyclical graphs and building agent runtimes that can handle complex reasoning tasks and flexible decision-making.'